In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

In [2]:
# Setup splinter
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Downloading: 100%|█████████████████| 8.82M/8.82M [00:00<00:00, 31.2MB/s]


In [24]:
#search page
# browser = Browser('firefox')
url = 'https://sfbay.craigslist.org/search/off?postal=94107&query=pdr&search_distance=4#search=1~gallery~0~0'
header={}
browser.visit(url)
html = browser.html
website = soup(html, 'html.parser')

In [25]:
#collect all of the search results

results = website.find_all("div", class_="gallery-card")


In [26]:
# for x in results:
#     if x.find(class_='meta') is not None:
#         info = x.find(class_='meta').text
        
# info

'1/30·potrero hill'

In [27]:
results_info = []

for x in results:
    if x.find(class_='titlestring') is not None:
        title = x.find(class_='titlestring').text
        link = x.find(class_='titlestring')['href']
    else: title = None
    if x.find(class_='meta') is not None:
        info = x.find(class_='meta').text
    else: info = None
    if x.find(class_='priceinfo') is not None:
        price = x.find(class_='priceinfo').text
    else: price = None
    table = {
        "title":title,
        "link":link,
        "info":info,
        "price":price,
    }
    results_info.append(table)

In [28]:
#present the results in a table
df = pd.DataFrame(results_info)
df

,title,link,info,price
0,For Lease // Newly Renovated Legal Office/PDR ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/28·SOMA / south beach,None
1,For Lease: Prime Retail / PDR Space With Flexi...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/27·12500ft2·mission district,None
2,For Lease // Newly Renovated Legal Office/PDR ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/6·SOMA / south beach,None
3,For Sale | Industrial Building w/ Secured Lot,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/28·potrero hill,None
4,For Lease // Brand New Industrial Building w/ ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/28·potrero hill,None
5,For Lease | Brick & Timber Warehouse - Cheap!!...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/28·mission district,None
6,For Lease // Prime Industrial Warehouse w/ Ons...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/24·bayview,None
7,FOR LEASE: Ground Floor Office / Flex Space,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/10·1250ft2·mission district,"$4,000"
8,For Lease // Penthouse Office in Potrero Hill ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/6·potrero hill,None
9,For Sale | Industrial Complex w/ Secured Parking,https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/30·potrero hill,None


In [31]:
#clean up our table
df["info"] = df["info"].replace('[^a-zA-Z0-9/ -]+', "-", regex=True)
df[["date", "sqft","neighborhood"]] = df["info"].str.split("-",expand=True)
df.head()

,title,link,info,price,date,sqft,neighborhood
0,For Lease // Newly Renovated Legal Office/PDR ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/28-SOMA / south beach,None,2/28,SOMA / south beach,None
1,For Lease: Prime Retail / PDR Space With Flexi...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/27-12500ft2-mission district,None,2/27,12500ft2,mission district
2,For Lease // Newly Renovated Legal Office/PDR ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/6-SOMA / south beach,None,2/6,SOMA / south beach,None
3,For Sale | Industrial Building w/ Secured Lot,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/28-potrero hill,None,2/28,potrero hill,None
4,For Lease // Brand New Industrial Building w/ ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,2/28-potrero hill,None,2/28,potrero hill,None


In [15]:
#across multiple pages

url = 'https://sfbay.craigslist.org/search/off?postal=94107&query=industrial&search_distance=4#search=1~gallery~0~0'
browser.visit(url)

for x in range(1,4):
    html = browser.html
    website = soup(html, 'html.parser')
    results = website.find_all(class_="cl-search-result cl-search-view-mode-gallery")

    all_results_info = []

    for x in results:
        if x.find(class_='titlestring') is not None:
            title = x.find(class_='titlestring').text
            link = x.find(class_='titlestring')['href']
        else: title = None
        if x.find(class_='meta') is not None:
            info = x.find(class_='meta').text
        else: info = None
        if x.find(class_='priceinfo') is not None:
            price = x.find(class_='priceinfo').text
        else: price = None
        browser.find_by_css(".cl-next-page").click()

        dictionary = {
            "title":title,
            "link":link,
            "info":info,
            "price":price,
        }
        all_results_info.append(dictionary)
        

In [16]:
pd.DataFrame(all_results_info)

,title,link,info,price
0,"3,000-7,000 SF Union St. Retail Space with Exp...",https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/31·marina / cow hollow,None
1,UPPER MARKET CORNER RETAIL OPPORTUNITY,https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/31·castro / upper market,None
2,FOR SALE SOMA BRICK & TIMBER RESTAURANT,https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/31·SOMA / south beach,None
3,Charming Jewel Space in Fillmore Street,https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/31·marina / cow hollow,None
4,Pacific Heights Retail on Best Block of Upper ...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/31·pacific heights,None
5,Ground Floor Commercial Space Available in Bus...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/31·Upper Market,None
6,Retail/Service Opportunity in Duboce Triangle,https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/31·castro / upper market,None
7,In the Heart of the Busiest Neighborhoods,https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/31·mission district,None
8,Retail Space Inside the Bustling Streets of Ma...,https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/31·castro / upper market,None
9,Recently Remodeled Pacific Heights Retail,https://sfbay.craigslist.org/sfc/off/d/san-fra...,1/31·pacific heights,None


In [38]:
#going into an individual listing

url = 'https://sfbay.craigslist.org/search/off?postal=94107&query=pdr&search_distance=4#search=1~gallery~0~0'
header={}
browser.visit(url)
html = browser.html
website = soup(html, 'html.parser')

results = website.find_all("div", class_="gallery-card")

for x in results:
    browser.find_by_css(".titlestring").click()
    if x.find(class_='postingtitle') is not None:
        title = x.find(class_='postingtitle').text
    else: title = None
    if x.find(class_='mapaddress') is not None:
        x.address = find(class_='mapaddress').text
        x.link = find(class_='mapaddress')['href']
    else: address = None
    browser.find_by_css(".backup").click()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.177)
Stacktrace:
0   chromedriver                        0x0000000101e1f138 chromedriver + 4923704
1   chromedriver                        0x0000000101d979d3 chromedriver + 4368851
2   chromedriver                        0x00000001019e0787 chromedriver + 472967
3   chromedriver                        0x00000001019b6c15 chromedriver + 302101
4   chromedriver                        0x0000000101a534cf chromedriver + 943311
5   chromedriver                        0x0000000101a682c6 chromedriver + 1028806
6   chromedriver                        0x0000000101a4dd23 chromedriver + 920867
7   chromedriver                        0x0000000101a14a4b chromedriver + 686667
8   chromedriver                        0x0000000101a16044 chromedriver + 692292
9   chromedriver                        0x0000000101dea8fe chromedriver + 4708606
10  chromedriver  

In [23]:
browser.quit()